## Char-RNN with embedding 古诗词

用 Char-RMM model 训练古诗词，model with embedding layer。参考code # http://blog.topspeedsnail.com/archives/10542

In [1]:
import collections
import numpy as np
import tensorflow as tf

In [2]:
poetry_file ='poetry.txt'
 
# 诗集
poetrys = []
with open(poetry_file, "r", encoding='utf-8',) as f:
	for line in f:
		try:
			title, content = line.strip().split(':')
			content = content.replace(' ','')
			if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content:
				continue
			if len(content) < 5 or len(content) > 79:
				continue
			content = '[' + content + ']'
			poetrys.append(content)
		except Exception as e: 
			pass
 
# 按诗的字数排序
poetrys = sorted(poetrys,key=lambda line: len(line))
print('唐诗总数: ', len(poetrys))

唐诗总数:  34646


In [3]:
# 统计每个字出现次数
all_words = []
for poetry in poetrys:
	all_words += [word for word in poetry]
counter = collections.Counter(all_words)
count_pairs = sorted(counter.items(), key=lambda x: -x[1])
words, _ = zip(*count_pairs)
 
# 取前多少个常用字
words = words[:len(words)] + (' ',)
# 每个字映射为一个数字ID
word_num_map = dict(zip(words, range(len(words))))
# 把诗转换为向量形式，参考TensorFlow练习1
to_num = lambda word: word_num_map.get(word, len(words))
poetrys_vector = [ list(map(to_num, poetry)) for poetry in poetrys]

# 每次取64首诗进行训练
batch_size = 64
n_chunk = len(poetrys_vector) // batch_size
x_batches = []
y_batches = []
for i in range(n_chunk):
	start_index = i * batch_size
	end_index = start_index + batch_size
 
	batches = poetrys_vector[start_index:end_index]
	length = max(map(len,batches))
	xdata = np.full((batch_size,length), word_num_map[' '], np.int32)
	for row in range(batch_size):
		xdata[row,:len(batches[row])] = batches[row]
	ydata = np.copy(xdata)
	ydata[:,:-1] = xdata[:,1:]
	"""
	xdata             ydata
	[6,2,4,6,9]       [2,4,6,9,9]
	[1,4,2,8,5]       [4,2,8,5,5]
	"""
	x_batches.append(xdata)
	y_batches.append(ydata)
 

In [4]:
#---------------------------------------RNN--------------------------------------#
 
input_data = tf.placeholder(tf.int32, [batch_size, None])
output_targets = tf.placeholder(tf.int32, [batch_size, None])
# 定义RNN
def neural_network(model='lstm', rnn_size=128, num_layers=2):
	if model == 'rnn':
		cell_fun = tf.nn.rnn_cell.BasicRNNCell
	elif model == 'gru':
		cell_fun = tf.nn.rnn_cell.GRUCell
	elif model == 'lstm':
		cell_fun = tf.nn.rnn_cell.BasicLSTMCell
 
	cell = cell_fun(rnn_size, state_is_tuple=True)
	cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
 
	initial_state = cell.zero_state(batch_size, tf.float32)
 
	with tf.variable_scope('rnnlm'):
		softmax_w = tf.get_variable("softmax_w", [rnn_size, len(words)+1])
		softmax_b = tf.get_variable("softmax_b", [len(words)+1])
		with tf.device("/gpu:0"):
			embedding = tf.get_variable("embedding", [len(words)+1, rnn_size])
			inputs = tf.nn.embedding_lookup(embedding, input_data)
    
    
	outputs, last_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
	output = tf.reshape(outputs,[-1, rnn_size])
 
	logits = tf.matmul(output, softmax_w) + softmax_b
	probs = tf.nn.softmax(logits)
	return logits, last_state, probs, cell, initial_state
#训练
def train_neural_network():
	logits, last_state, _, _, _ = neural_network()
	targets = tf.reshape(output_targets, [-1])   
	loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [targets], [tf.ones_like(targets, dtype=tf.float32)], len(words))
	cost = tf.reduce_mean(loss)
	learning_rate = tf.Variable(0.0, trainable=False)
	tvars = tf.trainable_variables()
	grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 5)
	optimizer = tf.train.AdamOptimizer(learning_rate)
	train_op = optimizer.apply_gradients(zip(grads, tvars))
 
	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())

		saver = tf.train.Saver(tf.all_variables())

		for epoch in range(553):
			sess.run(tf.assign(learning_rate, 0.002 * (0.97 ** epoch)))
			n = 0
			for batche in range(n_chunk):
				train_loss, _ , _ = sess.run([cost, last_state, train_op], feed_dict={input_data: x_batches[n], output_targets: y_batches[n]})
				n += 1
				print(epoch, batche, train_loss)
			if epoch % 7 == 0:
				saver.save(sess, 'checkpoint for CharRNN/poetry.module', global_step=epoch)


In [5]:
train_neural_network()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from checkpoint for CharRNN/poetry.module-413
413 0 2.94272
413 1 3.04692
413 2 3.37669
413 3 3.04902
413 4 3.59899
413 5 3.21108
413 6 3.58032
413 7 3.5685
413 8 3.21343
413 9 3.58814
413 10 3.59164
413 11 3.52522
413 12 3.45478
413 13 3.50752
413 14 3.4145
413 15 3.60719
413 16 3.50793
413 17 3.63204
413 18 3.6107
413 19 3.40696
413 20 3.42085
413 21 3.55173
413 22 3.46099
413 23 3.66016
413 24 3.51252
413 25 3.59694
413 26 3.536
413 27 3.51219
413 28 3.68415
413 29 3.47786
413 30 3.57558
413 31 3.62853
413 32 3.6461
413 33 3.61557
413 34 3.66689
413 35 3.65366
413 36 3.63278
413 37 3.52493
413 38 3.66644
413 39 3.5178
413 40 3.68202
413 41 3.58116
413 42 3.89138
413 43 3.54729
413 44 3.87405
413 45 3.76069
413 46 3.68264
413 47 3.39575
413 48 3.52583
413 49 3.76756
413 50 3.53899
413 51 3.72943
413 52 3.5391

413 486 4.20626
413 487 4.14275
413 488 4.32931
413 489 4.27368
413 490 4.30889
413 491 4.19491
413 492 4.19715
413 493 4.3529
413 494 4.30814
413 495 4.51164
413 496 4.36609
413 497 4.36999
413 498 4.33218
413 499 4.40018
413 500 4.40984
413 501 4.66875
413 502 4.65174
413 503 4.23395
413 504 4.44584
413 505 4.45484
413 506 4.45357
413 507 4.52166
413 508 4.48993
413 509 4.30005
413 510 4.37886
413 511 4.31559
413 512 4.44877
413 513 4.64292
413 514 4.35948
413 515 4.6092
413 516 4.43989
413 517 4.40441
413 518 4.34462
413 519 4.23908
413 520 4.42851
413 521 4.45055
413 522 4.47761
413 523 4.42884
413 524 4.72807
413 525 4.60366
413 526 4.41776
413 527 4.45815
413 528 4.47289
413 529 4.46135
413 530 4.37011
413 531 4.45406
413 532 4.41756
413 533 4.49287
413 534 4.46969
413 535 4.54457
413 536 4.38808
413 537 4.39776
413 538 4.44529
413 539 4.48882
413 540 4.82593
414 0 2.94272
414 1 3.04692
414 2 3.37669
414 3 3.04902
414 4 3.59899
414 5 3.21107
414 6 3.58032
414 7 3.5685
414 8 3.213

414 468 4.34263
414 469 4.32401
414 470 4.2419
414 471 4.26357
414 472 4.37024
414 473 4.27554
414 474 4.08604
414 475 3.9953
414 476 4.32047
414 477 4.4472
414 478 4.44772
414 479 4.44809
414 480 4.52782
414 481 4.45196
414 482 4.52809
414 483 4.411
414 484 4.19027
414 485 4.29498
414 486 4.20626
414 487 4.14275
414 488 4.32931
414 489 4.27368
414 490 4.30889
414 491 4.19492
414 492 4.19715
414 493 4.3529
414 494 4.30814
414 495 4.51164
414 496 4.36609
414 497 4.37
414 498 4.33218
414 499 4.40018
414 500 4.40984
414 501 4.66875
414 502 4.65173
414 503 4.23395
414 504 4.44584
414 505 4.45484
414 506 4.45357
414 507 4.52166
414 508 4.48993
414 509 4.30005
414 510 4.37886
414 511 4.31558
414 512 4.44877
414 513 4.64292
414 514 4.35948
414 515 4.6092
414 516 4.43989
414 517 4.40441
414 518 4.34462
414 519 4.23907
414 520 4.4285
414 521 4.45055
414 522 4.47761
414 523 4.42884
414 524 4.72807
414 525 4.60366
414 526 4.41776
414 527 4.45814
414 528 4.47289
414 529 4.46135
414 530 4.37011
414

415 450 4.60802
415 451 4.53562
415 452 4.39438
415 453 4.1689
415 454 4.26522
415 455 4.35363
415 456 4.34764
415 457 4.4264
415 458 4.1858
415 459 4.16183
415 460 4.26942
415 461 4.23569
415 462 4.28395
415 463 4.20283
415 464 4.39172
415 465 4.3498
415 466 4.42135
415 467 4.38317
415 468 4.34263
415 469 4.32401
415 470 4.2419
415 471 4.26357
415 472 4.37024
415 473 4.27554
415 474 4.08604
415 475 3.9953
415 476 4.32047
415 477 4.4472
415 478 4.44772
415 479 4.44809
415 480 4.52782
415 481 4.45196
415 482 4.52809
415 483 4.411
415 484 4.19027
415 485 4.29498
415 486 4.20626
415 487 4.14275
415 488 4.32931
415 489 4.27368
415 490 4.30889
415 491 4.19492
415 492 4.19715
415 493 4.3529
415 494 4.30814
415 495 4.51164
415 496 4.36609
415 497 4.37
415 498 4.33218
415 499 4.40018
415 500 4.40984
415 501 4.66875
415 502 4.65173
415 503 4.23395
415 504 4.44583
415 505 4.45484
415 506 4.45356
415 507 4.52165
415 508 4.48993
415 509 4.30005
415 510 4.37885
415 511 4.31558
415 512 4.44877
415 5

416 432 4.07239
416 433 4.10857
416 434 3.97923
416 435 4.3145
416 436 4.4124
416 437 4.17727
416 438 4.09633
416 439 4.27662
416 440 4.38762
416 441 4.49548
416 442 4.26919
416 443 4.40128
416 444 4.0822
416 445 3.97548
416 446 3.95532
416 447 4.14091
416 448 4.19772
416 449 4.47946
416 450 4.60802
416 451 4.53562
416 452 4.39438
416 453 4.1689
416 454 4.26522
416 455 4.35363
416 456 4.34764
416 457 4.4264
416 458 4.1858
416 459 4.16183
416 460 4.26942
416 461 4.23569
416 462 4.28395
416 463 4.20283
416 464 4.39172
416 465 4.3498
416 466 4.42135
416 467 4.38317
416 468 4.34263
416 469 4.32401
416 470 4.2419
416 471 4.26357
416 472 4.37024
416 473 4.27554
416 474 4.08604
416 475 3.9953
416 476 4.32047
416 477 4.4472
416 478 4.44772
416 479 4.44809
416 480 4.52782
416 481 4.45196
416 482 4.52809
416 483 4.41101
416 484 4.19027
416 485 4.29498
416 486 4.20626
416 487 4.14275
416 488 4.32931
416 489 4.27368
416 490 4.30889
416 491 4.19492
416 492 4.19715
416 493 4.3529
416 494 4.30814
416

...

557 186 4.08256
557 187 4.19253
557 188 4.1735
557 189 4.08836
557 190 4.142
557 191 4.23997
557 192 4.11737
557 193 4.18152
557 194 4.13092
557 195 4.07281
557 196 3.96378
557 197 4.04578
557 198 3.88923
557 199 4.07182
557 200 3.94025
557 201 3.8559
557 202 3.81729
557 203 4.00725
557 204 4.04193
557 205 3.66937
557 206 3.61774
557 207 3.63026
557 208 3.90726
557 209 3.96548
557 210 3.98348
557 211 3.93756
557 212 4.04783
557 213 3.9491
557 214 4.06196
557 215 4.20514
557 216 3.95566
557 217 3.96036
557 218 3.90771
557 219 4.03131
557 220 3.86908
557 221 3.85925
557 222 3.98382
557 223 4.11149
557 224 4.1369
557 225 4.05487
557 226 4.17109
557 227 4.15979
557 228 4.16409
557 229 4.0334
557 230 3.88747
557 231 3.79297
557 232 3.92904
557 233 3.93664
557 234 3.63827
557 235 3.75877
557 236 3.85572
557 237 4.04536
557 238 3.84832
557 239 3.88333
557 240 3.93084
557 241 3.76785
557 242 3.76802
557 243 4.02255
557 244 3.9266
557 245 3.94368
557 246 3.71987
557 247 3.88551
557 248 3.90759


558 171 3.38838
558 172 3.55982
558 173 3.91804
558 174 4.22138
558 175 4.4848
558 176 4.12267
558 177 3.68901
558 178 3.70556
558 179 3.75651
558 180 3.74367
558 181 4.13291
558 182 4.08045
558 183 4.02408
558 184 3.9463
558 185 3.89458
558 186 4.08256
558 187 4.19253
558 188 4.1735
558 189 4.08836
558 190 4.142
558 191 4.23997
558 192 4.11737
558 193 4.18152
558 194 4.13092
558 195 4.07281
558 196 3.96378
558 197 4.04578
558 198 3.88923
558 199 4.07182
558 200 3.94025
558 201 3.8559
558 202 3.81729
558 203 4.00725
558 204 4.04193
558 205 3.66937
558 206 3.61774
558 207 3.63026
558 208 3.90726
558 209 3.96548
558 210 3.98348
558 211 3.93756
558 212 4.04783
558 213 3.9491
558 214 4.06196
558 215 4.20514
558 216 3.95566
558 217 3.96036
558 218 3.90771
558 219 4.03131
558 220 3.86908
558 221 3.85925
558 222 3.98382
558 223 4.11149
558 224 4.1369
558 225 4.05487
558 226 4.17109
558 227 4.15979
558 228 4.16409
558 229 4.0334
558 230 3.88747
558 231 3.79297
558 232 3.92904
558 233 3.93664
5

559 153 3.8215
559 154 3.96289
559 155 4.0328
559 156 3.83013
559 157 3.80458
559 158 3.9279
559 159 3.66852
559 160 3.78929
559 161 3.9449
559 162 3.93822
559 163 4.11922
559 164 4.10053
559 165 4.10415
559 166 3.93648
559 167 3.70483
559 168 3.43223
559 169 3.52292
559 170 3.78147
559 171 3.38838
559 172 3.55982
559 173 3.91804
559 174 4.22138
559 175 4.4848
559 176 4.12267
559 177 3.68901
559 178 3.70556
559 179 3.75651
559 180 3.74367
559 181 4.13291
559 182 4.08045
559 183 4.02408
559 184 3.9463
559 185 3.89458
559 186 4.08256
559 187 4.19253
559 188 4.1735
559 189 4.08836
559 190 4.142
559 191 4.23997
559 192 4.11737
559 193 4.18152
559 194 4.13092
559 195 4.07281
559 196 3.96378
559 197 4.04578
559 198 3.88923
559 199 4.07182
559 200 3.94025
559 201 3.8559
559 202 3.81729
559 203 4.00725
559 204 4.04193
559 205 3.66937
559 206 3.61774
559 207 3.63026
559 208 3.90726
559 209 3.96548
559 210 3.98348
559 211 3.93756
559 212 4.04783
559 213 3.9491
559 214 4.06196
559 215 4.20514
559

560 135 3.66192
560 136 3.92233
560 137 3.85638
560 138 3.92775
560 139 4.35956
560 140 4.11619
560 141 4.00417
560 142 3.71034
560 143 3.85782
560 144 3.74432
560 145 3.76645
560 146 4.03067
560 147 3.65143
560 148 3.63903
560 149 3.69308
560 150 3.91188
560 151 3.79036
560 152 3.82819
560 153 3.8215
560 154 3.96289
560 155 4.0328
560 156 3.83013
560 157 3.80458
560 158 3.9279
560 159 3.66852
560 160 3.78929
560 161 3.9449
560 162 3.93822
560 163 4.11922
560 164 4.10053
560 165 4.10415
560 166 3.93648
560 167 3.70483
560 168 3.43223
560 169 3.52292
560 170 3.78147
560 171 3.38838
560 172 3.55982
560 173 3.91804
560 174 4.22138
560 175 4.4848
560 176 4.12267
560 177 3.68901
560 178 3.70556
560 179 3.75651
560 180 3.74367
560 181 4.13291
560 182 4.08045
560 183 4.02408
560 184 3.9463
560 185 3.89458
560 186 4.08256
560 187 4.19253
560 188 4.1735
560 189 4.08836
560 190 4.142
560 191 4.23997
560 192 4.11737
560 193 4.18152
560 194 4.13092
560 195 4.07281
560 196 3.96378
560 197 4.04578
5

KeyboardInterrupt: 

In [5]:
batch_size = 1
n_chunk = len(poetrys_vector) // batch_size
x_batches = []
y_batches = []
for i in range(n_chunk):
	start_index = i * batch_size
	end_index = start_index + batch_size
 
	batches = poetrys_vector[start_index:end_index]
	length = max(map(len,batches))
	xdata = np.full((batch_size,length), word_num_map[' '], np.int32)
	for row in range(batch_size):
		xdata[row,:len(batches[row])] = batches[row]
	ydata = np.copy(xdata)
	ydata[:,:-1] = xdata[:,1:]
	"""
	xdata             ydata
	[6,2,4,6,9]       [2,4,6,9,9]
	[1,4,2,8,5]       [4,2,8,5,5]
	"""
	x_batches.append(xdata)
	y_batches.append(ydata)
 
 
#---------------------------------------RNN--------------------------------------#
 
input_data = tf.placeholder(tf.int32, [batch_size, None])
output_targets = tf.placeholder(tf.int32, [batch_size, None])

In [6]:
def gen_poetry():
	def to_word(weights):
		t = np.cumsum(weights)
		s = np.sum(weights)
		sample = int(np.searchsorted(t, np.random.rand(1)*s))
		return words[sample]
 
	_, last_state, probs, cell, initial_state = neural_network()
 
	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())
 
		saver = tf.train.Saver(tf.all_variables())
		saver.restore(sess, 'checkpoint for CharRNN/poetry.module-553')
 
		state_ = sess.run(cell.zero_state(1, tf.float32))
 
		x = np.array([list(map(word_num_map.get, '['))])
		[probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})
		word = to_word(probs_)
		#word = words[np.argmax(probs_)]
		poem = ''
		while word != ']':
			poem += word
			x = np.zeros((1,1))
			x[0,0] = word_num_map[word]
			[probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})
			word = to_word(probs_)
			#word = words[np.argmax(probs_)]
		return poem
 
print(gen_poetry())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from checkpoint for CharRNN/poetry.module-553
早岁调于大，安于寄一枝。紫氛抛卜照，地鹊近平光。招讴癖绶禄，焕为松氏新。


In [7]:

def gen_poetry_with_head(head):
	def to_word(weights):
		t = np.cumsum(weights)
		s = np.sum(weights)
		sample = int(np.searchsorted(t, np.random.rand(1)*s))
		return words[sample]
 
	_, last_state, probs, cell, initial_state = neural_network()
 
	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())
 
		saver = tf.train.Saver(tf.all_variables())
		saver.restore(sess, 'checkpoint for CharRNN/poetry.module-28')
 
		state_ = sess.run(cell.zero_state(1, tf.float32))
		poem = ''
		i = 0
		for word in head:
			while word != '，' and word != '。':
				poem += word
				x = np.array([list(map(word_num_map.get, word))])
				[probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})
				word = to_word(probs_)
			if i % 2 == 0:
				poem += '，'
			else:
				poem += '。'
			i += 1
		return poem


print(gen_poetry_with_head('一二三四'))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from checkpoint for CharRNN/poetry.module-28
一家看蜀北西，二乡。三君接旬，四主使已。
